In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import os

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

win_path = f'../features/4_winner/*.gz'
fname=''
# submit = pd.read_csv('../input/sample_submission.csv')
submit = []

HOME = os.path.expanduser('~')

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

2018-12-31 08:20:10,768 utils 366 [INFO]    [logger_func] start 


In [2]:
aft = pd.read_csv('../valid/1230_115_lgb__feat152_CV3.654551433862739_lr0.01.csv')
bef = pd.read_csv('../valid/1230_102_lgb__feat151_CV3.653822391857942_lr0.01.csv')

aft.head()

,feature,0_importance,1_importance,2_importance,3_importance,4_importance,avg_importance,rank
0,111_ker_new_hist_purchase_amount_max@,1521,1174,1925,1770,1526,1583.2,1
1,117_ah1_auth1_hist_month_diff_mean@,1461,933,2296,1550,1499,1547.8,2
2,118_ah0_auth0_hist_month_diff_mean@,1434,812,2236,1613,1403,1499.6,3
3,111_ker_new_hist_purchase_date_uptonow@,1183,714,1656,1280,1059,1178.4,4
4,120_ath_diff_last_first_authorized_flag_mean@,980,675,1628,1106,967,1071.2,5


In [3]:
df = aft.merge(bef, how='left', on='feature')
df.head()

,feature,0_importance_x,1_importance_x,2_importance_x,3_importance_x,4_importance_x,avg_importance_x,rank_x,0_importance_y,1_importance_y,2_importance_y,3_importance_y,4_importance_y,avg_importance_y,rank_y
0,111_ker_new_hist_purchase_amount_max@,1521,1174,1925,1770,1526,1583.2,1,2095.0,1106.0,2118.0,1659.0,1514.0,1698.4,3.0
1,117_ah1_auth1_hist_month_diff_mean@,1461,933,2296,1550,1499,1547.8,2,2449.0,801.0,2699.0,1344.0,1362.0,1731.0,1.0
2,118_ah0_auth0_hist_month_diff_mean@,1434,812,2236,1613,1403,1499.6,3,2430.0,689.0,2681.0,1421.0,1323.0,1708.8,2.0
3,111_ker_new_hist_purchase_date_uptonow@,1183,714,1656,1280,1059,1178.4,4,1940.0,605.0,1929.0,1128.0,975.0,1315.4,4.0
4,120_ath_diff_last_first_authorized_flag_mean@,980,675,1628,1106,967,1071.2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['diff_importance'] = df['avg_importance_x'] - df['avg_importance_y']
df['diff_rank'] = df['rank_x'] - df['rank_y']

In [11]:
df[['feature', 'rank_x', 'rank_y', 'diff_rank', 'diff_importance']].sort_values(by='diff_importance')

,feature,rank_x,rank_y,diff_rank,diff_importance
2,118_ah0_auth0_hist_month_diff_mean@,3,2.0,1.0,-209.2
1,117_ah1_auth1_hist_month_diff_mean@,2,1.0,1.0,-183.2
9,119_ath_auth1_0_hist_purchase_date_min_diff@,10,7.0,3.0,-178.2
8,119_ath_auth1_0_hist_purchase_date_max_diff@,9,6.0,3.0,-175.6
10,111_ker_hist_purchase_date_min@,11,9.0,2.0,-160.8
19,111_ker_hist_month_lag_var@,20,17.0,3.0,-145.2
18,111_ker_hist_authorized_flag_mean@,19,16.0,3.0,-141.6
3,111_ker_new_hist_purchase_date_uptonow@,4,4.0,0.0,-137.0
25,111_ker_hist_first_buy@,26,20.0,6.0,-124.8
7,117_ah1_auth1_hist_purchase_amount_mean@,8,10.0,-2.0,-124.6
